In [ ]:
import re

def clean_sql_code(self, sql_code: str) -> str:
    if not sql_code:
        return ""

    match = re.search(
        r"```sql\s*(.*?)\s*```",
        sql_code,
        flags=re.IGNORECASE | re.DOTALL
    )

    if match:
        return match.group(1).strip()

    sql_code = re.sub(r"```.*?```", "", sql_code, flags=re.DOTALL)
    sql_code = re.sub(r"^\s*sql\s*\n", "", sql_code, flags=re.IGNORECASE)

    return sql_code.strip()


In [7]:
text = """
Here is the SQL query that retrieves the audio introduction (or video) for the point of interest (POI) "000":

```sql
SELECT TOP 1 
    SSDL.SubProjectID, 
    SSDL.SubProjectName AS Name, 
    SSDL.Location, 
    MDG.MediaType, 
    MDG.MediaURL, 
    MDG.LanguageID 
FROM {DB_PREFIX}.SubProjects AS SSDL 
LEFT JOIN {DB_PREFIX}.SubprojectAttractions AS A ON SSDL.SubProjectID = A.SubProjectID  
LEFT JOIN {DB_PREFIX}.SubprojectAttractionDetails AS D ON A.SubProjectAttractionID = D.SubProjectAttractionID  
LEFT JOIN {DB_PREFIX}.SubprojectAttractionDetailsMedia AS MDG ON D.SubProjectAttractionDetailID = MDG.SubProjectAttractionDetailID  
WHERE 
    SSDL.POI = '000' 
    OR A.POI = '000' 
    OR D.POI = '000' 
    AND 
        MDG.MediaType = 'video' 
ORDER BY NEWID();
```
This query joins the relevant tables to retrieve the media information for POI "000," ensuring that the media type is 'video' (which, in this system, encompasses both audio and video media). The query may return a video or audio file, depending on what's available.
"""

In [ ]:
a = clean_sql_code(sql_code = text)
a

"SELECT TOP 1 \n    SSDL.SubProjectID, \n    SSDL.SubProjectName AS Name, \n    SSDL.Location, \n    MDG.MediaType, \n    MDG.MediaURL, \n    MDG.LanguageID \nFROM {DB_PREFIX}.SubProjects AS SSDL \nLEFT JOIN {DB_PREFIX}.SubprojectAttractions AS A ON SSDL.SubProjectID = A.SubProjectID  \nLEFT JOIN {DB_PREFIX}.SubprojectAttractionDetails AS D ON A.SubProjectAttractionID = D.SubProjectAttractionID  \nLEFT JOIN {DB_PREFIX}.SubprojectAttractionDetailsMedia AS MDG ON D.SubProjectAttractionDetailID = MDG.SubProjectAttractionDetailID  \nWHERE \n    SSDL.POI = '000' \n    OR A.POI = '000' \n    OR D.POI = '000' \n    AND \n        MDG.MediaType = 'video' \nORDER BY NEWID();"

: 

In [5]:
class TextRequest(BaseModel):
    text: str

In [ ]:
class GCStorage:
    def __init__(self, credentials_path: str | None = None):
        if credentials_path:
            os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path

        self.client = storage.Client()

    def upload_blob(self, bucket_name, src_file_name):
        bucket = self.client.bucket(bucket_name)
        blob_name = os.path.basename(src_file_name)
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(src_file_name)
        blob.make_public()
        return blob.public_url
    
    def delete_blob(self, url: str):
        """
        Delete a blob from its public URL or gs:// path.
        Args:
            url: The public URL (https://storage.googleapis.com/...) or gs:// path.
        """
        try:
            # Parse bucket name and blob name from URL
            if url.startswith("gs://"):
                # Example: gs://my-bucket/tts/audio.mp3
                parts = url.replace("gs://", "").split("/", 1)
                bucket_name, blob_name = parts[0], parts[1]
            else:
                # Example: https://storage.googleapis.com/my-bucket/tts/audio.mp3
                parsed = urlparse(url)
                parts = parsed.path.lstrip("/").split("/", 1)
                bucket_name, blob_name = parts[0], parts[1]

            bucket = self.client.bucket(bucket_name)
            blob = bucket.blob(blob_name)
            blob.delete()

        except exceptions.NotFound:
            print(f"File not found: gs://{bucket_name}/{blob_name}")

        except Exception as e:
            print(f"Delete error: {e}")


In [18]:
a = GCStorage(credentials_path="./GCP/rare-karma-468001-k3-a7efcf8a7c80.json")
a.upload_blob("tts-script" ,"./GCP/1.jpg")

'https://storage.googleapis.com/tts-script/1.jpg'

In [20]:
a.delete_blob('https://storage.googleapis.com/tts-script/1.jpg')

NotFound: 404 DELETE https://storage.googleapis.com/storage/v1/b/tts-script/o/1.jpg?prettyPrint=false: No such object: tts-script/1.jpg

In [9]:
import os

In [ ]:
from google import genai
from google.genai.types import GenerateContentConfig, Modality, Part, ImageConfig
from PIL import Image
from io import BytesIO

def generate_multi_style_image(
    content_uri: str,
    style_uris: list[str],
    prompt: str = (
        "Generate an image that keeps the composition of the content image "
        "but adopts the color palette and lighting style of the reference images."
    ),
    aspect_ratio: str = "1:1",
    model_id: str = "gemini-2.0-flash",
    output_path: str = "styled_result2.png",
):
    """
    Generate an image based on one content image and multiple style reference images using Vertex AI (Gemini/Imagen).
    
    Args:
        content_uri: GCS URI of the main content image.
        style_uris: List of GCS URIs for style reference images.
        prompt: Instruction describing how to combine content and styles.
        aspect_ratio: Output aspect ratio ("1:1", "16:9", etc.)
        model_id: The Gemini or Imagen model to use.
        output_path: Local path to save the generated image.
    """
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./GCP/guidepassasiacloud-8493e3dfd4c1.json"
    client = genai.Client(vertexai=True, project='guidepassasiacloud', location='us-central1')

    # Build the "contents" parts list with descriptive text
    contents = [
        "This is the **content image** whose composition should be followed:",
        Part.from_uri(file_uri=content_uri, mime_type="image/jpeg"),
        "These are **style reference images** whose colors and lighting should be followed:",
    ]
    for uri in style_uris:
        contents.append(Part.from_uri(file_uri=uri, mime_type="image/jpeg"))

    # Add final instruction text
    contents.append(prompt)

    # Generate
    response = client.models.generate_content(
        model=model_id,
        contents=contents,
        config=GenerateContentConfig(
            response_modalities=[Modality.IMAGE],
            image_config=ImageConfig(aspect_ratio=aspect_ratio),
        ),
    )

    # Parse response
    for part in response.candidates[0].content.parts:
        img = Image.open(BytesIO(part.inline_data.data))

        # Convert sang bytes
        buffer = BytesIO()
        img.save(buffer, format="PNG")
        buffer.seek(0)

        # Upload
        gcs = GCStorage()
        url = gcs.upload_from_bytes(bucket_name=GCStorage, buffer.getvalue(), f"{uuid.uuid4()}.png")

    return output_path


In [27]:
generate_multi_style_image(
    content_uri="https://storage.googleapis.com/guidepassasia_chatbot/1.jpg",
    style_uris=[
        "https://storage.googleapis.com/guidepassasia_chatbot/e137b9da-7278-49d6-8e68-1fe7183c513e.png",
    ],
    aspect_ratio="16:9",
    prompt="Keep the subject and framing of the content image, "
           "but make it look like it was painted in the style of the reference images."
)


ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'Multi-modal output is not supported.', 'status': 'INVALID_ARGUMENT'}}

In [28]:
from Database.db import MultiDBManager

In [30]:
db = MultiDBManager()

In [31]:
rows = db.run_query(3, "SELECT TOP 10 * FROM Attractions")

AttributeError: 'MultiDBManager' object has no attribute 'run_query'

In [1]:
from sqlalchemy import text

class DBWrapper:
    """Gói SQLAlchemy engine thành interface có run_query()"""
    def __init__(self, engine):
        self.engine = engine

    def run_query(self, sql: str, params: dict = None):
        """
        Thực thi câu SQL và trả kết quả dạng list[dict]
        """
        try:
            with self.engine.connect() as conn:
                result = conn.execute(text(sql), params or {})
                return [dict(r._mapping) for r in result]
        except Exception as e:
            print(f"[DBWrapper] ⚠️ Query failed: {e}")
            return []


In [5]:
from Database.db import MultiDBManager

# Khởi tạo DB manager
dbm = MultiDBManager()

# Lấy engine theo region (ví dụ region 3)
engine = dbm.get_engine(3)

# Gói engine vào DBWrapper
db = DBWrapper(engine)

# Chạy query
rows = db.run_query("SELECT TOP 10 * FROM pro92766_proguidepass.SubProjectAttractions")
for row in rows:
    print(row)


[DBManager] ✅ Created engine for region 3: 112.78.2.156
{'SubProjectAttractionID': 6, 'POI': '406', 'SubProjectID': 4, 'AttractionName': 'Eiffel Tower', 'AttractionImage': '/images/2dc6fa16-f8bf-47f8-89e5-8e7f3e95d4c8_1721397.jpg', 'Introduction': 'The Eiffel Tower, Paris’s most iconic landmark, rises gracefully above the city skyline as a symbol of elegance, innovation, and romance. Completed in 1889 for the World’s Fair, this iron masterpiece by Gustave Eiffel was once considered controversial, but today it is the very heart of Paris. From its base, you can gaze up at the intricate lattice of steel that seems to dance with light, and from its summit, the entire city unfolds like a breathtaking panorama—bridges crossing the Seine, grand boulevards, and rooftops stretching toward the horizon.\r\n\r\nDay or night, the Eiffel Tower offers a magical experience. By day, it gleams under the Parisian sun; by night, it transforms into a beacon of light, sparkling every hour in a dazzling show

[DBManager] 💤 Disposing idle engine for region 3 (112.78.2.156)
